The purpose of this function is to take a vessel and determine at which axial distance at which another vessel branches off.
Additionally it needs to check to see what the terminus condition of the vessel is and then create the appropriate ending conditions for it whether that be flowing directly into another vessel (continous), branching into multiple vessels (split) or no end condition in which case we will need to apply the 0D end condition (tree). 

Our first necessary step will be to import the excel file which contains all of the file names and the links between them

In [8]:
#Install 
#!pip install pandas

In [2]:
#Import
import pandas as pd
import numpy as np
import scipy
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [48]:
#Display two point cloud with different colours in one o3d window
def display_two_pointclouds(array_1,array_2,windowname):
    p1_pcd = o3d.geometry.PointCloud()
    p1_pcd.points = o3d.utility.Vector3dVector(array_1)
    p1_pcd.paint_uniform_color([1, 0.706, 0])


    p2_pcd = o3d.geometry.PointCloud()
    p2_pcd.points = o3d.utility.Vector3dVector(array_2)
    p2_pcd.paint_uniform_color([0, 0.706, 1])

    concate_pc = np.concatenate((array_1, array_2),axis = 0)
    p1_color = np.asarray(p1_pcd.colors)
    p2_color = np.asarray(p2_pcd.colors)
    p3_color = np.concatenate((p1_color,p2_color), axis=0)

    p3_pcd = o3d.geometry.PointCloud()
    p3_pcd.points = o3d.utility.Vector3dVector(concate_pc)
    p3_pcd.colors = o3d.utility.Vector3dVector(p3_color)
    o3d.visualization.draw_geometries([p3_pcd],window_name = windowname)

## Import Excel File

In [11]:
arteries_sheet = pd.read_excel('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FlowTracker.xlsx', sheet_name = 0)
veins_sheet = pd.read_excel('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FlowTracker.xlsx', sheet_name = 1)

We will need to pass the vessel name when this is a function but for now we will us arteries_lleg5 flowing with lleg_2 branching off it



In [12]:
vessel_file_name = 'arteries_lleg5'

In [13]:
#Is it an artery or vein?
if 'art' or 'aorta' in vessel_file_name:
    sheet = arteries_sheet
else: 
    sheet = veins_sheet

In [20]:
#Find the associated row of the vessel within the sheet
vessel_row = sheet[sheet['Filename'].str.match(vessel_file_name)].index.values[0]

#Check for end condition
end_point = sheet.at[sheet.index[vessel_row],'End Point']

#Parse meaning
if pd.isna(end_point):
    end_condition = 'tree'
elif ',' in end_point:
    end_condition = 'split'
else:
    end_condition = 'continous'
    
#Determine if any vessels branch off at any point
out_flow = sheet.at[sheet.index[vessel_row],'Out Flow']

if pd.isna(out_flow):
    branch = 'none'
else:
    branch = out_flow.split(',')



['arteries_lleg2']


## Deal with branches

In [4]:
#Find the nearest points for 
if branch != 'none':
    
    #Find the file names of the main and branches
    main_branch_filename = vessel_file_name + '_fitted_data.npy'
    branch_filenames = []
    distance_of_split = []
    
    for i in range(0,len(branch)):
        branch_name = branch[i]
        vessel_row = sheet[sheet['Anatomy Name'].str.match(branch_name)].index.values[0]
        branch_filename = sheet.at[sheet.index[vessel_row],'Filename']
        branch_filename = branch_filename + '_fitted_data.npy'
        branch_filenames.append(branch_filename)
        
    #Now we import all of the vessel file 
    main_branch_array = np.load('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FittedVesselsFiles\\' + main_branch_filename)
    
    for i in range(0,len(branch)):
        branch_filename = branch_filenames[i]
        
        branch_array = np.load('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FittedVesselsFiles\\' + branch_filename)
        
    #Find the nearest points
        dist_array = scipy.spatial.distance.cdist(main_branch_array[:,0:3],branch_array[:,0:3])
        dist_array = dist_array[:,0]
        
        index = np.where (np.min(dist_array) == dist_array)[0]
        
        
        #Find the distance along the main vessel axis that this split occurs
        distance = 0
        

        for j in range(0,index[0]-1):
            dist = np.linalg.norm(main_branch_array[i,0:3] - main_branch_array[i+1,0:3])
            distance = distance + dist
        distance_of_split.append(distance)   
        
else:
    distance_of_split = []


SyntaxError: invalid syntax (2530381717.py, line 9)

# Convert Existing Excel Document into a segmented vessel array data frame for calculation purposes!

In [3]:
df = pd.DataFrame(columns=['Name','Centre Axis Array', 'Radius Array','Total Length','End Condition'])



,Name,Centre Axis Array,Radius Array,End Condition
